<!-- Stand-alone notebook? -->


# Deflection of a membrane
<div id="ftut:poisson:membrane"></div>

The previous problem and code targeted a simple test problem where we
can easily verify the implementation. Now we turn the attention to
a more physically relevant problem, in a non-trivial geometry, and
that results in solutions of somewhat more exciting shape.

We want to compute the deflection $D(x,y)$ of a two-dimensional,
circular membrane, subject to a load $p$ over the membrane. The
appropriate PDE model is

<!-- Equation labels as ordinary links -->
<div id="_auto1"></div>

$$
\begin{equation}
-T\nabla^2 D = p(x,y)\quad\hbox{in }\Omega = \{ (x,y)\,|\, x^2+y^2\leq R\}{\thinspace .}
\label{_auto1} \tag{1}
\end{equation}
$$

Here, $T$ is the tension in the membrane (constant), and $p$ is the external
pressure load.
The boundary of the membrane has no
deflection, implying $D=0$ as boundary condition.
A localized load can be modeled as a Gaussian function:

<!-- Equation labels as ordinary links -->
<div id="_auto2"></div>

$$
\begin{equation}
p(x,y) = {A\over 2\pi\sigma}\exp{\left(
- {1\over2}\left( {x-x_0\over\sigma}\right)^2
- {1\over2}\left( {y-y_0\over\sigma}\right)^2
\right)}\, .
\label{_auto2} \tag{2}
\end{equation}
$$

The parameter $A$ is the amplitude of the pressure,
$(x_0,y_0)$ the localization of the maximum point of the load,
and $\sigma$ the "width" of $p$.

## Scaling

The localization of the pressure, $(x_0,y_0)$, is for simplicity
set to $(0, R_0)$.
There are many physical parameters in this problem, and we can benefit
from grouping them by means of scaling. Let us introduce dimensionless
coordinates $\bar x = x/R$, $\bar y = y/R$, and a dimensionless
deflection $w=D/D_c$, where $D_c$ is a characteristic size of the
deflection. Introducing $\bar R_0=R_0/R$, we get

$$
\frac{\partial^2 w}{\partial\bar x^2} +
\frac{\partial^2 w}{\partial\bar y^2}= \alpha
\exp{\left(
- \beta^2(\bar x^2
+ (\bar y-\bar R_0)^2)\right)}\hbox{ for } \bar x^2 + \bar y^2 < 1,
$$

where

$$
\alpha = \frac{R^2A}{2\pi T D_c\sigma},\quad\beta = \frac{R}{\sqrt{2}\sigma}{\thinspace .}
$$

With an appropriate scaling, $\bar w$ and its derivatives are of size
unity, so the left-hand side of the scaled PDE is about unity in size,
while the right-hand side has $\alpha$ as its characteristic size.
This suggest choosing $\alpha$ to be unity, or around unit.  We shall
in particular choose $\alpha=4$. With this value, the solution is
$w(\bar x,\bar y) = 1-\bar x^2 - \bar y^2$.  (One can also find the
analytical solution in scaled coordinates and show that the maximum
deflection $D(0,0)$ is $D_c$ if we choose $\alpha=4$ to determine
$D_c$.)  With $D_c=AR^2/(8\pi\sigma T)$ and dropping the bars we get
the scaled problem

<!-- Equation labels as ordinary links -->
<div id="ftut:poisson1:membrane:scaled:eq"></div>

$$
\begin{equation}
\nabla^2w = 4\exp{\left(
- \beta^2(x^2
+ (y-R_0)^2)\right)},
\label{ftut:poisson1:membrane:scaled:eq} \tag{3}
\end{equation}
$$

to be solved over the unit circle with $w=0$ on the boundary.  Now
there are only two parameters to vary: the dimensionless extent of the
pressure, $\beta$, and the localization of the pressure peak, $R_0\in
[0,1]$.  As $\beta\rightarrow 0$, we have a special case with solution
$w=1-x^2-y^2$.

Given a computed $w$, the physical deflection is given by

$$
D = \frac{AR^2}{8\pi\sigma T}w{\thinspace .}
$$

Just a few modifications are necessary in our previous program to solve
this new problem.

## Defining the mesh

A mesh over the unit circle can be created by the `mshr` tool in
FEniCS:

In [1]:
from fenics import *

In [2]:
from mshr import *
domain = Circle(Point(0.0, 0.0), 1.0)
n = 20
mesh = generate_mesh(domain, n)

The `Circle` shape from `mshr` takes the center and radius of the circle
as the two first arguments, while `n` is the resolution, here the
suggested number of cells per radius.

## Defining the load


The right-hand side pressure function
is represented by an `Expression` object. There
are two physical parameters in the formula for $f$ that enter the
expression string and these parameters must have their values set
by keyword arguments:

In [3]:
beta = 8
R0 = 0.6
p = Expression(
    '4*exp(-pow(beta,2)*(pow(x[0], 2) + pow(x[1]-R0, 2)))',
    beta=beta, R0=R0)

The coordinates in `Expression` objects *must* be a vector
with indices 0, 1, and 2, and with the name `x`. Otherwise
we are free to introduce names of parameters as long as these are
given default values by keyword arguments. All the parameters
initialized by keyword arguments can at any time have their
values modified. For example, we may set

```Python
        p.beta = 12
        p.R0 = 0.3
```

We also need to define a function space and boundary conditions:

In [4]:
V = FunctionSpace(mesh, 'P', 2)

def u0_boundary(x, on_boundary):
    return on_boundary

u0 = Constant(0)
bc = DirichletBC(V, u0, u0_boundary)

## Variational form

We may introduce `w` instead of `u` as primary unknown and `p` instead
of `f` as right-hand side function:

In [5]:
w = TrialFunction(V)
v = TestFunction(V)
a = dot(grad(w), grad(v))*dx
L = p*v*dx

w = Function(V)
solve(a == L, w, bc)

## Curve plots through the domain

<!-- Nice with some matplotlib plots in notebooks...and very useful for -->
<!-- scientific investigations. -->

The best way to compare the load and the deflection is to make a curve plot
along the line $y=0$. This is just a matter of defining a set of points
along the line and evaluating the finite element functions `w` and `p`
at these points:

In [6]:
%matplotlib inline

# Curve plot along x=0 comparing p and w
import numpy as np
import matplotlib.pyplot as plt
tol = 1E-8  # Avoid hitting points outside domain
y = np.linspace(-1+tol, 1-tol, 101)
points = [(0, y_) for y_ in y]  # 2D points
w_line = np.array([w(point) for point in points])
p_line = np.array([p(point) for point in points])
plt.plot(y, 100*w_line, 'r-', y, p_line, 'b--') # magnify w
plt.legend(['100 x deflection', 'load'], loc='upper left')
plt.xlabel('y');  plt.ylabel('$p$ and $100u$')
plt.savefig('tmp1.pdf');  plt.savefig('tmp1.png')

[hpl 1: Clash between `plot` and Matplotlib. Must click quit on the built-in
plots before Matplotlib will plot. Maybe take out `plot` from notebook?]